In [1]:
# COMEX Silver Inventory Analysis & Plotting (Plotly)
# --------------------------------------------------
# Fix: prevent header commodity name like "SILVER" from being parsed as a depository.

import os
import re
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# =========================
# 1. 参数设置
# =========================
DATA_DIR = r"E:\code\python\mytools\SystemMacro\CME\CME_Stocks\Silver"
FILE_PATTERN = re.compile(r"Silver_Stocks_(\d{8})\.xls$", re.IGNORECASE)

# =========================
# 2. 读取单个 CME Silver 文件
# =========================
def read_single_file(file_path: str, date_str: str) -> pd.DataFrame:
    """
    返回 tidy dataframe:
    Date | Depository | Category | Ounces
    Category ∈ {Registered, Eligible, Total}
    """
    df = pd.read_excel(file_path, header=None)

    records = []
    current_depository = None

    # 这些关键词/表头行不应该当成仓库
    HARD_EXCLUDE = {
        "SILVER",
        "COMMODITY EXCHANGE, INC.",
        "COMMODITY EXCHANGE, INC",
        "METAL DEPOSITORY STATISTICS",
        "TROY OUNCE",
        "DEPOSITORY",
    }

    # 任何包含这些 token 的全大写行也排除（防止 TOTAL 行、表尾声明等）
    INVALID_TOKENS = [
        "TOTAL", "REGISTERED", "ELIGIBLE", "COMBINED",
        "COMMODITY", "EXCHANGE", "INC", "METAL", "STATISTICS", "REPORT", "ACTIVITY"
    ]

    for _, row in df.iterrows():
        cell0 = str(row.iloc[0]).strip() if len(row) > 0 else ""
        if not cell0 or cell0.lower() == "nan":
            continue

        # ===== 仓库名识别 =====
        # CME 仓库名一般是全大写；但要排除表头/合计/说明等全大写行
        if cell0.isupper():
            if cell0 in HARD_EXCLUDE:
                continue
            if "DEPOSITORY" in cell0:  # 像 "ASAHI DEPOSITORY LLC" 这种是仓库名，允许；但 "DEPOSITORY" 单独表头排除已在 HARD_EXCLUDE
                pass
            if any(tok in cell0 for tok in INVALID_TOKENS):
                # 注意：这里会排除 "TOTAL REGISTERED" 等行
                # 但不会误杀 "ASAHI DEPOSITORY LLC"（不含 TOTAL/REGISTERED/ELIGIBLE/COMBINED 等）
                if cell0 not in ["ASAHI DEPOSITORY LLC", "INTERNATIONAL DEPOSITORY SERVICES OF DELAWARE", "CNT DEPOSITORY, INC."]:
                    # 为避免误杀个别包含通用 token 的合法仓库名，上面做了少量白名单兜底
                    # 你如遇到被误杀的仓库名，把它加进白名单即可
                    continue

            # 进一步排除纯品类名（例如偶尔出现的 "SILVER"）
            if cell0 in {"SILVER"}:
                continue

            current_depository = cell0
            continue

        # ===== 类别行识别 =====
        if cell0 in {"Registered", "Eligible", "Total"} and current_depository:
            # TOTAL TODAY 通常在最右列；用“最后一个非空值”更稳健
            try:
                last_val = row.dropna().iloc[-1]
                ounces = float(last_val)
            except Exception:
                continue

            records.append({
                "Date": pd.to_datetime(date_str, format="%Y%m%d"),
                "Depository": current_depository,
                "Category": cell0,
                "Ounces": ounces
            })

    out = pd.DataFrame(records)

    # 兜底清洗：若仍出现 SILVER 这种伪仓库，直接删除并告警
    if not out.empty and (out["Depository"] == "SILVER").any():
        print(f"[WARN] Found invalid Depository='SILVER' in {os.path.basename(file_path)}. Dropping.")
        out = out[out["Depository"] != "SILVER"].copy()

    return out


# =========================
# 3. 批量读取所有文件
# =========================
all_dfs = []
for fname in os.listdir(DATA_DIR):
    m = FILE_PATTERN.match(fname)
    if not m:
        continue
    date_str = m.group(1)
    fpath = os.path.join(DATA_DIR, fname)
    print(f"Reading {fname}")
    one = read_single_file(fpath, date_str)
    if not one.empty:
        all_dfs.append(one)

if not all_dfs:
    raise RuntimeError(f"No valid Silver_Stocks_YYYYMMDD.xls files parsed from: {DATA_DIR}")

df_all = pd.concat(all_dfs, ignore_index=True)

# 二次兜底（全局）
df_all = df_all[df_all["Depository"].str.upper().ne("SILVER")].copy()

# =========================
# 4. 汇总到全市场层面
# =========================
market_df = (
    df_all
    .groupby(["Date", "Category"], as_index=False)["Ounces"]
    .sum()
)

pivot_market = (
    market_df
    .pivot(index="Date", columns="Category", values="Ounces")
    .sort_index()
)

# 防止偶发缺列
for col in ["Registered", "Eligible", "Total"]:
    if col not in pivot_market.columns:
        pivot_market[col] = pd.NA

pivot_market["Registered_Ratio"] = pivot_market["Registered"] / pivot_market["Total"]

pivot_market["dRegistered"] = pivot_market["Registered"].diff()
pivot_market["dEligible"] = pivot_market["Eligible"].diff()
pivot_market["dTotal"] = pivot_market["Total"].diff()

# =========================
# 5. 图 1：库存多 Y 轴折线（Registered / Eligible / Total）
# =========================
fig_inventory = go.Figure()

fig_inventory.add_trace(go.Scatter(
    x=pivot_market.index, y=pivot_market["Registered"],
    name="Registered", yaxis="y1"
))
fig_inventory.add_trace(go.Scatter(
    x=pivot_market.index, y=pivot_market["Eligible"],
    name="Eligible", yaxis="y2"
))
fig_inventory.add_trace(go.Scatter(
    x=pivot_market.index, y=pivot_market["Total"],
    name="Total", yaxis="y3"
))

fig_inventory.update_layout(
    title="COMEX Silver Inventory (Multi Y-Axis)",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Registered (oz)", side="left"),
    yaxis2=dict(title="Eligible (oz)", overlaying="y", side="right"),
    yaxis3=dict(title="Total (oz)", overlaying="y", side="right", position=0.95),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
fig_inventory.show()

# =========================
# 6. 图 2：Registered / Total Ratio
# =========================
fig_ratio = px.line(
    pivot_market, y="Registered_Ratio",
    title="Registered / Total Ratio"
)
fig_ratio.show()

# =========================
# 7. 图 3：Daily Net Change（分组柱状）
# =========================
fig_flow = go.Figure()
fig_flow.add_bar(x=pivot_market.index, y=pivot_market["dRegistered"], name="Δ Registered")
fig_flow.add_bar(x=pivot_market.index, y=pivot_market["dEligible"], name="Δ Eligible")
fig_flow.add_bar(x=pivot_market.index, y=pivot_market["dTotal"], name="Δ Total")
fig_flow.update_layout(barmode="group", title="Daily Inventory Net Change (oz)")
fig_flow.show()

# =========================
# 8. 图 4：Registered 按仓库堆叠面积图
# =========================
reg_df = df_all[df_all["Category"] == "Registered"].copy()
reg_df = (
    reg_df.groupby(["Date", "Depository"], as_index=False)["Ounces"].sum()
)

fig_stack = px.area(
    reg_df, x="Date", y="Ounces", color="Depository",
    title="Registered Silver by Depository"
)
fig_stack.show()

# =========================
# 9. 图 5：ΔRegistered 热力图
# =========================
reg_pivot = (
    reg_df.pivot(index="Date", columns="Depository", values="Ounces")
    .sort_index()
)
reg_diff = reg_pivot.diff()

fig_heat = px.imshow(
    reg_diff.T,
    aspect="auto",
    color_continuous_scale="RdBu",
    title="Δ Registered by Depository (Heatmap)"
)
fig_heat.show()

# =========================
# 10. 打印关键数字
# =========================
print("\n=== Market-level Inventory (last 10 days) ===")
print(pivot_market[["Registered", "Eligible", "Total", "Registered_Ratio"]].tail(10).round(6))

print("\n=== Daily Net Changes (last 10 days) ===")
print(pivot_market[["dRegistered", "dEligible", "dTotal"]].tail(10))

latest_date = reg_df["Date"].max()
print(f"\n=== Latest Registered by Depository ({latest_date.date()}) ===")
print(
    reg_df[reg_df["Date"] == latest_date]
    .sort_values("Ounces", ascending=False)
    .reset_index(drop=True)
)

print("\n=== Top 5 Depository ΔRegistered (last day) ===")
if latest_date in reg_diff.index:
    last_diff = reg_diff.loc[latest_date].dropna()
    if not last_diff.empty:
        print("\nTop 5 increases:")
        print(last_diff.sort_values(ascending=False).head(5))
        print("\nTop 5 decreases:")
        print(last_diff.sort_values(ascending=True).head(5))
    else:
        print("No ΔRegistered data available for latest date.")
else:
    print("Latest date not found in reg_diff index.")


Reading Silver_Stocks_20251204.xls
Reading Silver_Stocks_20251205.xls
Reading Silver_Stocks_20251208.xls
Reading Silver_Stocks_20251209.xls
Reading Silver_Stocks_20251210.xls
Reading Silver_Stocks_20251211.xls
Reading Silver_Stocks_20251212.xls
Reading Silver_Stocks_20251215.xls
Reading Silver_Stocks_20251216.xls
Reading Silver_Stocks_20251217.xls
Reading Silver_Stocks_20251218.xls
Reading Silver_Stocks_20251219.xls
Reading Silver_Stocks_20251222.xls
Reading Silver_Stocks_20251223.xls



=== Market-level Inventory (last 10 days) ===
Category      Registered      Eligible         Total  Registered_Ratio
Date                                                                  
2025-12-10  1.367415e+08  3.196737e+08  4.564152e+08          0.299599
2025-12-11  1.378020e+08  3.180151e+08  4.558171e+08          0.302319
2025-12-12  1.375818e+08  3.157780e+08  4.533598e+08          0.303472
2025-12-15  1.298554e+08  3.247050e+08  4.545604e+08          0.285672
2025-12-16  1.301479e+08  3.236982e+08  4.538461e+08          0.286767
2025-12-17  1.306070e+08  3.223438e+08  4.529508e+08          0.288347
2025-12-18  1.303244e+08  3.228518e+08  4.531762e+08          0.287580
2025-12-19  1.286167e+08  3.251102e+08  4.537269e+08          0.283467
2025-12-22  1.286167e+08  3.220268e+08  4.506435e+08          0.285407
2025-12-23  1.272143e+08  3.236658e+08  4.508801e+08          0.282147

=== Daily Net Changes (last 10 days) ===
Category    dRegistered    dEligible       dTotal
Date     